In [2]:
!pip install --upgrade gensim -q
!pip install matplotlib -q
!pip install spacy
import gensim
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import spacy
import string
import gensim.downloader as api
print(list(gensim.downloader.info()['models'].keys()))

2023-05-10 20:38:42.504929: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-10 20:38:42.766229: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-10 20:38:42.767627: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-10 20:38:44.028718: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-10 20:38:45.662438: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [3]:
wv = api.load('glove-wiki-gigaword-50')

In [4]:
wv['one']

array([ 0.31474 ,  0.41662 ,  0.1348  ,  0.15854 ,  0.88812 ,  0.43317 ,
       -0.55916 ,  0.030476, -0.14623 , -0.14273 , -0.17949 , -0.17343 ,
       -0.49264 ,  0.26775 ,  0.48799 , -0.29537 ,  0.18485 ,  0.14937 ,
       -0.75009 , -0.35651 , -0.23699 ,  0.1849  ,  0.17237 ,  0.23611 ,
        0.14077 , -1.9031  , -0.65353 , -0.022539,  0.10383 , -0.43705 ,
        3.781   , -0.044077, -0.046643,  0.027274,  0.51883 ,  0.13353 ,
        0.23231 ,  0.25599 ,  0.060888, -0.065618, -0.15556 ,  0.30818 ,
       -0.093586,  0.33296 , -0.14613 ,  0.016332, -0.24251 , -0.20526 ,
        0.07009 , -0.11568 ], dtype=float32)

In [5]:
data = pd.read_csv('imdb.csv')



In [6]:
import string

In [143]:
data_x = np.array(data['review'])
data_y = np.array(data['sentiment'])
x = []
print(data_x.shape)
print(data_y.shape)
for i in range(len(data_x)):
    translating = str.maketrans('', '', string.punctuation)
    z = data_x[i].translate(translating)
    l1 = np.array(z.split())
    l2 = l1[np.where(l1!='<br /><br />')]
    x.append(l2)
    # print(l2.shape)
    
xdata = np.array(x)


(50000,)
(50000,)


/tmp/ipykernel_57839/3149578955.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  xdata = np.array(x)


In [144]:
n_dim = 50
finalembed = np.zeros((xdata.shape[0],n_dim))
for i in range(xdata.shape[0]):
    embedvect = np.zeros((xdata[i].shape[0],n_dim))
    for j in range(xdata[i].shape[0]):
        try:
            word = xdata[i][j].lower()
            embedvect[j] = wv[word]
        except KeyError as ke:
            #eror catched
            pass
    finalembed[i] = np.mean(embedvect,axis=0)
print(finalembed.shape)
    


        

(50000, 50)


In [145]:
ydata = np.zeros(data_y.shape[0])
res = np.where(data_y=='positive')
ydata[res] = 1

In [79]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler


In [147]:
# train_data, test_data, train_labels, test_labels = finalembed[:40000], finalembed[40000:], ydata[:40000], ydata[40000:]
train_data = finalembed[:40000]
train_labels = ydata[:40000]
scaler = StandardScaler()
scaler.fit(train_data)
train_data = scaler.transform(train_data)

In [82]:
model = MLPClassifier(hidden_layer_sizes=(30,20,5), max_iter=200, batch_size=50 , alpha=1e-5,learning_rate_init=0.001,learning_rate='adaptive', solver='adam', verbose=True)

In [148]:
model.fit(train_data, train_labels)

Iteration 1, loss = 0.51921216
Iteration 2, loss = 0.48394119
Iteration 3, loss = 0.47594020
Iteration 4, loss = 0.47039113
Iteration 5, loss = 0.46650761
Iteration 6, loss = 0.46254527
Iteration 7, loss = 0.45985665
Iteration 8, loss = 0.45708824
Iteration 9, loss = 0.45521222
Iteration 10, loss = 0.45373452
Iteration 11, loss = 0.45137811
Iteration 12, loss = 0.44959419
Iteration 13, loss = 0.44872088
Iteration 14, loss = 0.44699563
Iteration 15, loss = 0.44586330
Iteration 16, loss = 0.44418600
Iteration 17, loss = 0.44274853
Iteration 18, loss = 0.44220628
Iteration 19, loss = 0.44054526
Iteration 20, loss = 0.43922773
Iteration 21, loss = 0.43883617
Iteration 22, loss = 0.43733620
Iteration 23, loss = 0.43621225
Iteration 24, loss = 0.43620881
Iteration 25, loss = 0.43459643
Iteration 26, loss = 0.43415301
Iteration 27, loss = 0.43332835
Iteration 28, loss = 0.43245472
Iteration 29, loss = 0.43197961
Iteration 30, loss = 0.43105802
Iteration 31, loss = 0.43052178
Iteration 32, los

MLPClassifier(alpha=1e-05, batch_size=50, hidden_layer_sizes=(30, 20, 5),
              learning_rate='adaptive', verbose=True)

In [149]:
print('TRAIN DATA')
pred_y = model.predict(train_data)
c = 0
t = 0
# print(pred_y)
# print(test_labels)
for i in range(len(pred_y)):
    t += 1
    if(pred_y[i]==train_labels[i]):
        c+=1
accq = c/t

print(accq)

TRAIN DATA
0.81945


In [85]:
print('VAL DATA')
pred_y = model.predict(test_data)
c = 0
t = 0
# print(pred_y)
# print(test_labels)
for i in range(len(pred_y)):
    t += 1
    if(pred_y[i]==test_labels[i]):
        c+=1
accq = c/t

print(accq)

VAL DATA
0.5735


In [40]:
import json

In [86]:
debaised_wv = json.load(open('data.json'))


In [87]:
for el in debaised_wv.keys():
    debaised_wv[el] = np.array(debaised_wv[el])


In [150]:
model_debias = MLPClassifier(hidden_layer_sizes=(30,20,5), max_iter=200, batch_size=50 , alpha=1e-5,learning_rate_init=0.001,learning_rate='adaptive', solver='adam', verbose=True)

In [151]:
n_dim = 50
finalembed_db = np.zeros((xdata.shape[0],n_dim))
for i in range(xdata.shape[0]):
    embedvect_db = np.zeros((xdata[i].shape[0],n_dim))
    for j in range(xdata[i].shape[0]):
        try:
            word = xdata[i][j].lower()
            embedvect_db[j] = debaised_wv[word]
        except KeyError as ke:
            #eror catched
            pass
    finalembed_db[i] = np.mean(embedvect_db,axis=0)
print(finalembed_db.shape)

(50000, 50)


In [152]:
train_data_db, test_data_db, train_labels, test_labels = finalembed_db[:40000], finalembed_db[40000:], ydata[:40000], ydata[40000:]
scaler = StandardScaler()
scaler.fit(train_data_db)
train_data_db = scaler.transform(train_data_db)

In [154]:
model_debias.fit(train_data_db, train_labels)

Iteration 1, loss = 0.53594505
Iteration 2, loss = 0.48898505
Iteration 3, loss = 0.47997840
Iteration 4, loss = 0.47423896
Iteration 5, loss = 0.47036754
Iteration 6, loss = 0.46693767
Iteration 7, loss = 0.46394848
Iteration 8, loss = 0.46186112
Iteration 9, loss = 0.45948825
Iteration 10, loss = 0.45789440
Iteration 11, loss = 0.45590634
Iteration 12, loss = 0.45428116
Iteration 13, loss = 0.45193328
Iteration 14, loss = 0.45110388
Iteration 15, loss = 0.44938964
Iteration 16, loss = 0.44865356
Iteration 17, loss = 0.44762729
Iteration 18, loss = 0.44619990
Iteration 19, loss = 0.44482684
Iteration 20, loss = 0.44388540
Iteration 21, loss = 0.44217512
Iteration 22, loss = 0.44150031
Iteration 23, loss = 0.44071223
Iteration 24, loss = 0.43986036
Iteration 25, loss = 0.43865772
Iteration 26, loss = 0.43784146
Iteration 27, loss = 0.43735355
Iteration 28, loss = 0.43579862
Iteration 29, loss = 0.43569561
Iteration 30, loss = 0.43469512
Iteration 31, loss = 0.43346964
Iteration 32, los

/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(alpha=1e-05, batch_size=50, hidden_layer_sizes=(30, 20, 5),
              learning_rate='adaptive', verbose=True)

In [155]:
print('TRAIN DATA')
pred_y_db = model_debias.predict(train_data_db)
c = 0
t = 0
# print(pred_y)
# print(test_labels)
for i in range(len(pred_y_db)):
    t += 1
    if(pred_y_db[i]==train_labels[i]):
        c+=1
accq2 = c/t
print(pred_y_db)
print(accq2)

TRAIN DATA
[0. 1. 1. ... 1. 0. 1.]
0.8223


In [104]:
print(xdata[49])
print(xdata[50])
print(xdata[6])
#Jake - Jane, BOOGEYMAN - BOOGEYWOMAN

['Average' 'and' 'surprisingly' 'tame' 'Fulci' 'giallo' 'which' 'means'
 'its' 'still' 'quite' 'bad' 'by' 'normal' 'standards' 'but' 'redeemed'
 'by' 'its' 'solid' 'buildup' 'and' 'some' 'nice' 'touches' 'such' 'as'
 'a' 'neat' 'time' 'twist' 'on' 'the' 'issues' 'of' 'visions' 'and'
 'clairvoyancebr' 'br' 'The' 'genres' 'wellknown' 'weaknesses' 'are' 'in'
 'full' 'gear' 'banal' 'dialogue' 'wooden' 'acting' 'illogical' 'plot'
 'points' 'And' 'the' 'finale' 'goes' 'on' 'much' 'too' 'long' 'while'
 'the' 'denouement' 'proves' 'to' 'be' 'a' 'rather' 'lame' 'or' 'shall'
 'I' 'say' 'limp' 'affairbr' 'br' 'Fulcis' 'ironic' 'handling' 'of'
 'giallo' 'norms' 'is' 'amusing' 'though' 'Yellow' 'clues' 'wherever'
 'you' 'lookbr' 'br' '3' 'out' 'of' '10' 'limping' 'killers']
['Return' 'to' 'the' '36th' 'Chamber' 'is' 'one' 'of' 'those' 'classic'
 'KungFu' 'movies' 'which' 'Shaw' 'produces' 'back' 'in' 'the' '70s' 'and'
 '80s' 'whose' 'genre' 'is' 'equivalent' 'to' 'the' 'spaghetti' 'westerns'
 'of' 

In [189]:
#changing the he/she bit
# 5, 6 only his
# 18 only he
# arr = np.array([3, 18,20, 21, 22,30, 31, 33,48, 50])
arr = np.arange(0,500)
newx = xdata.copy()
for i in arr:
    newx[i][np.where(xdata[i]=='boy')] = 'girl'
    newx[i][np.where(xdata[i]=='Jake')] = 'Jane'
    newx[i][np.where(xdata[i]=='BOOGEYMAN')] = 'BOOGEYWOMAN'
    newx[i][np.where(xdata[i]=='him')] = 'her'
    newx[i][np.where(xdata[i]=='his')] = 'her'
    newx[i][np.where(xdata[i]=='he')] = 'she'
    newx[i][np.where(xdata[i]=='He')] = 'She'
    newx[i][np.where(xdata[i]=='man')] = 'woman'
    newx[i][np.where(xdata[i]=='men')] = 'women'
    newx[i][np.where(xdata[i]=='His')] = 'Her'
    newx[i][np.where(xdata[i]=='himself')] = 'herself'
    newx[i] = np.array(newx[i])
    
print(newx.shape)
# print(newx[3].shape)
newlabel = ydata[arr]
cf_x = newx[arr]
print(cf_x[0])
wocf_x = train_data[arr]
print(wocf_x.shape)
# print(cf_x.shape)

(50000,)
['One' 'of' 'the' 'other' 'reviewers' 'has' 'mentioned' 'that' 'after'
 'watching' 'just' '1' 'Oz' 'episode' 'youll' 'be' 'hooked' 'They' 'are'
 'right' 'as' 'this' 'is' 'exactly' 'what' 'happened' 'with' 'mebr' 'br'
 'The' 'first' 'thing' 'that' 'struck' 'me' 'about' 'Oz' 'was' 'its'
 'brutality' 'and' 'unflinching' 'scenes' 'of' 'violence' 'which' 'set'
 'in' 'right' 'from' 'the' 'word' 'GO' 'Trust' 'me' 'this' 'is' 'not' 'a'
 'show' 'for' 'the' 'faint' 'hearted' 'or' 'timid' 'This' 'show' 'pulls'
 'no' 'punches' 'with' 'regards' 'to' 'drugs' 'sex' 'or' 'violence' 'Its'
 'is' 'hardcore' 'in' 'the' 'classic' 'use' 'of' 'the' 'wordbr' 'br' 'It'
 'is' 'called' 'OZ' 'as' 'that' 'is' 'the' 'nickname' 'given' 'to' 'the'
 'Oswald' 'Maximum' 'Security' 'State' 'Penitentary' 'It' 'focuses'
 'mainly' 'on' 'Emerald' 'City' 'an' 'experimental' 'section' 'of' 'the'
 'prison' 'where' 'all' 'the' 'cells' 'have' 'glass' 'fronts' 'and' 'face'
 'inwards' 'so' 'privacy' 'is' 'not' 'high' 'on' 

In [190]:
n_dim = 50
finalembed_3 = np.zeros((cf_x.shape[0],n_dim))
for i in range(cf_x.shape[0]):
    embedvect_2 = np.zeros((cf_x[i].shape[0],n_dim))
    for j in range(cf_x[i].shape[0]):
        try:
            word = cf_x[i][j].lower()
            embedvect_2[j] = debaised_wv[word]
        except KeyError as ke:
            #eror catched
            pass
    finalembed_3[i] = np.mean(embedvect_2,axis=0)
print(finalembed_3[2])
print(wocf_x[2])
print(finalembed_3.shape)

[ 0.28529094  0.13210342 -0.1394233  -0.16517263  0.39653932  0.20983482
 -0.37921092 -0.14194835 -0.25698245  0.08589549 -0.04460047  0.07389782
 -0.31964759 -0.02927432  0.42264622  0.12374204  0.05790066  0.07479913
 -0.41132248 -0.29700593 -0.03744637  0.24029389  0.2205694   0.05273097
  0.19822768 -1.46255916 -0.43493696  0.15621121  0.28452806 -0.21412682
  2.88537048  0.11739268 -0.10472235 -0.28086487  0.09447243 -0.0114608
  0.0150759   0.19966423 -0.088518   -0.26534821 -0.14675073  0.14202446
 -0.03911085  0.109966   -0.01707097  0.03986071 -0.06888796 -0.21988414
 -0.05819866  0.06628345]
[ 0.21283261  0.06703991 -0.3198739  -0.31303724  0.71230656  0.32955637
 -0.31169909 -0.51789001 -1.1357901   0.20113893 -0.05915901 -0.89251523
  0.03147797 -0.01899797 -0.19336613  0.05049628 -0.78760625  0.65997205
 -1.19508399  0.31838533 -0.54581947  0.16794917  0.13108796  0.11627786
 -0.35162208 -0.2595135   0.47687311  0.60845739  0.38641236  0.69732696
  0.06207218  0.04315387  

In [191]:
scaler = StandardScaler()
scaler.fit(finalembed_3)
finalembed_3 = scaler.transform(finalembed_3)

In [192]:
pred_1 = model.predict(wocf_x)

pred_3 = model.predict(finalembed_3)

print('True labels: ', newlabel)
print(pred_1)
# print(pred_2)
print(pred_3)
# print(pred_4)

True labels:  [1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 0. 1. 1. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0.
 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 1. 1. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1.
 0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0.
 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1.
 0. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0.
 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 1.
 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1.
 1. 0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0.
 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0.
 0. 1. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 1. 

In [187]:
from sklearn.metrics import accuracy_score

In [ ]:
pred_2 = model_debias.predict(wocf_x)
pred_4 = model_debias.predict(finalembed_3)
# print('True labels: ', newlabel)
# print(pred_1)
# print(pred_2)
# print(pred_3)
# print(pred_4)
print('Accuracy scores')
print('normal-original')
acc1 = accuracy_score(newlabel, pred_1)
print(acc1)
print('debias-original')
acc2 = accuracy_score(newlabel, pred_2)
print(acc2)
print('normal-cf')
acc3 = accuracy_score(newlabel, pred_3)
print(acc3)
print('debias-cf')
acc4 = accuracy_score(newlabel, pred_4)
print(acc4)